In [2]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile
import zipfile
import subprocess
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [3]:
PROJ_DIR = f'{WORK}/ADA_Project'
RAW_IMGS_DIR = f'{PROJ_DIR}/datasets/metfaces-release/images'
RESIZED_IMGS_DIR = f'{PROJ_DIR}/datasets/metfaces_resized_imgs'
DATA_CUSTOM_DIR = f'{PROJ_DIR}/datasets/metfaces_custom'
TRAIN_RUNS_DIR = f'{PROJ_DIR}/training_runs/metfaces_training-runs'

In [5]:
# # data was downloaded from https://drive.google.com/open?id=1w-Os4uERBmXwCm7Oo_kW6X3Sd2YHpJMC
raw_imgs = glob(f'{RAW_IMGS_DIR}/*')
len(raw_imgs)

1336

In [6]:
for x in tqdm(raw_imgs):
    DeepFAMS.preprocessing.resize_imgs(x, (512, 512), RESIZED_IMGS_DIR)

100%|██████████| 1336/1336 [04:36<00:00,  4.83it/s]


In [7]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 1336, Resized: 1336


In [8]:
DeepFAMS.preprocessing.tf_record_exporter(
    tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/metfaces_resized_imgs"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/metfaces_custom"
Added 1336 images.                      


In [9]:
with open('compile_model.sh', 'w') as f:
        f.write(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py \
--outdir={TRAIN_RUNS_DIR} \
--gpus=2 \
--data={DATA_CUSTOM_DIR} \
--snap=1 \
--kimg=1''')
        
DeepFAMS.utils.execute('cat compile_model.sh')

#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py --outdir=/work/chaselab/malyetama/ADA_Project/training_runs/metfaces_training-runs --gpus=2 --data=/work/chaselab/malyetama/ADA_Project/datasets/metfaces_custom --snap=1 --kimg=1


In [ ]:
p = subprocess.Popen("bash compile_model.sh",
                     shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

while p.poll() is None:
    line = p.stdout.readline()
    print(line)
    
os.remove('compile_model.sh')

In [10]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/metfaces_training-runs/00002-metfaces_custom-auto2-resumecustom/network-snapshot-000024.pkl


In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)